## Setting up the env variables

In [ ]:
# Import Azure OpenAI
from langchain.llms import AzureOpenAI
from langchain.chains import LLMChain
from dotenv import load_dotenv
load_dotenv("../../.env")
from langchain.chat_models import AzureChatOpenAI

## Intent chain - (Intent Skill)

In [ ]:
from langchain.schema import BaseOutputParser
from langchain.prompts import PromptTemplate

class IntentOutputParser(BaseOutputParser):
    """Parse the output of an LLM call to a comma-separated list."""

    def parse(self, text: str):
        """Parse the output of an LLM call."""
        intents = ["GET PARKING SPOT", "GET REGISTRATION DETAILS"]
        for i in intents:
            if i in text:
                return i
        return "INTENT UNCLEAR"


intents = ["GET PARKING SPOT", "GET REGISTRATION DETAILS"]
prompt_template = PromptTemplate.from_template("""You are a helpful assistant that identifies the intent of a user's query. 
There are 2 possible intents - GET PARKING SPOT, GET REGISTRATION DETAILS. 
Choose the most appropriate intent based on the human's query.
What is the intent for the following ask? 
{ask}
""")

llm = AzureOpenAI(
    deployment_name="completion-gpt-35-turbo",
    model_name="gpt-35-turbo",
    temperature=0.0
)
intent_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="intent", output_parser=IntentOutputParser(), verbose=True)

In [ ]:
intent_chain.run("Where should I park my car?")

### Few shot prompt template

In [ ]:
from langchain.prompts.chat import (
    ChatPromptTemplate
)
from langchain.prompts import FewShotChatMessagePromptTemplate

intent_identification_examples = [
    {"input": "Find me the nearest parking spot.", "output": "GET PARKING SPOT"},
    {"input": "Do you have the registration details of my vehicle?", "output": "GET REGISTRATION DETAILS"},
    {"input": "Where can I park my vehicle?", "output": "GET PARKING SPOT"},
    {"input": "Give me the registration details.", "output": "GET REGISTRATION DETAILS"}
]

# This is a prompt template used to format each individual example.
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=intent_identification_examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", 
         """You are a helpful assistant that identifies the intent of a user's query. 
         There are 2 possible intents - GET PARKING SPOT, GET REGISTRATION DETAILS. 
         Choose the most appropriate intent based on the human's query. 
         Here are some examples for you to understand how to output the intent."""
         ),
        few_shot_prompt,
        ("human", "{input}\nAI: "),
    ]
)

llm = AzureChatOpenAI(
    deployment_name="completion-gpt-35-turbo",
    model_name="gpt-35-turbo",
    temperature=0.0
)
intent_chain = LLMChain(llm=llm, prompt=final_prompt, output_key="intent", output_parser=IntentOutputParser(), verbose=True)

In [ ]:
intent_chain.run("Get the registeration details")

In [ ]:
import gradio as gr

def intent_chain_func(query):
    ans = intent_chain.run(query)
    return f"{ans}"

demo_intent_chain = gr.Interface(
    fn=intent_chain_func,
    inputs=["text"],
    outputs=["text"],
    title="Intent Recognizer"
)
demo_intent_chain.launch()

## Fetch user vehicle details

In [ ]:
# Fetching the vehicle details for the user
import pandas as pd

def get_vehicle_details_string(row):
    values = list(row)
    indices = row.index
    st = ""
    for a,b in list(zip(indices, values)):
        st += a + "-->" + b
        st += ", "
    return st

def fetch_vehicle_details_from_live_database(user):
    df_vehicle_details = pd.read_csv("./data/VehicleDetails.csv")
    df_user_vehicle = df_vehicle_details[df_vehicle_details["Employee"] == user]
    if len(df_user_vehicle) == 0:
        "NO REGISTERED VEHICLE FOUND"
    else:
        # return df_user_vehicle.iloc[0,:]["Outlook/Building Name"]
        return get_vehicle_details_string(df_user_vehicle.iloc[0,:])

In [ ]:
import gradio as gr

demo = gr.Interface(
    fn=fetch_vehicle_details_from_live_database,
    inputs=["text"],
    outputs=["text"],
    title="Get vehicle details"
)
demo.launch()

## Fetch available parking spots

In [ ]:
# Fetching the vehicle details for the user
import pandas as pd

def get_building_num(row):
    return row["Outlook/Building Name"]

def fetch_available_parking_spots_in_building(building):
    df_available_space = pd.read_csv("./data/AvailableSpace.csv")
    df_available_space = df_available_space[df_available_space["Building Number"] == building]
    if len(df_available_space) == 0:
        "NO PARKING SPOT FOUND"
    else:
        return list(df_available_space["Parking Spot Number"])

## Wrapping everything together

In [ ]:
from langchain.tools import BaseTool, Tool, tool
from pydantic import BaseModel, Field
from langchain.agents import create_csv_agent, AgentType, initialize_agent

def intent_recognizer_api(query: str) -> str:
    """Identifies the intent of the user's query"""
    ans = intent_chain.run(query)
    return ans

def get_employee_vehicle_details_api(user: str) -> str:
    # """Returns the details for an employee. The details contain the user's name, vehicle type, building name, fuel type of the vehicle, registration number, license plate number and the reserved parking."""
    """Returns the vehicle details for any employee. """
    ans = fetch_vehicle_details_from_live_database(user)
    return ans

def fetch_parking_spots_in_building_api(building_name: str) -> str:
    """There is a database containing all the available parking spots. This function performs a lookup on the database to identify the available parking spots in a specific building"""
    # ans = fetch_available_parking_spots_in_building(building_name)
    agent = create_csv_agent(
        llm,
        "./data/AvailableSpace.csv",
        verbose=True,
        agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    )
    ans = agent.run(f"Get me all the available parking spots in building {building_name}")
    return ans

def ask_user_for_clarification_api(query_for_user: str) -> str:
    # """Asks the user for any required information/input. You can only ask for the user's name, or ask them to choose a parking spot."""
    """Asks the user for any required information/input."""
    # print(query_for_user)
    input_from_user = input()
    return input_from_user




In [ ]:
t1 = Tool.from_function(
        func=intent_recognizer_api,
        name="get_intent_for_the_users_ask",
        description="Identifies the intent for the users ask"
)

t2 = Tool.from_function(
        func=get_employee_vehicle_details_api,
        name="get_employee_vehicle_details",
        description="Returns the employee, thier building details and their vehicle details given the user's name"
)

t3 = Tool.from_function(
        func=fetch_parking_spots_in_building_api,
        name="fetch_available_parking_spots_in_building",
        description="Performs a lookup on the database to identify the available parking spots in a specific building."
)

t4 = Tool.from_function(
        func=ask_user_for_clarification_api,
        name="ask_user_for_clarification",
        description="Asks the user for any required information/input."
)

agent = initialize_agent(
        [t2, t3, t4], 
    llm, 
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
    verbose=True)

In [ ]:
agent.run("Help me find a parking spot")